In [3]:
import getpass
import os

# Set LangSmith Environment Variables
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API key for LangSmith")


In [4]:
# Set up google llm model for RAG usage
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")


print("Initializing chat model!")
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")
print("Chat model initialization over!")

Initializing chat model!
Chat model initialization over!


In [5]:
print("Initializing embedder model!")
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
print("Embedder model initialized!")

Initializing embedder model!


/opt/miniconda3/envs/rag-agent-musings/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Embedder model initialized!


In [6]:
print("Initialize vector store!")
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
print("Vector store initialized!")

Initialize vector store!
Vector store initialized!


In [7]:
# Loading documents for RAG use

# get beautiful soup
import bs4
# get Document Loaders
from langchain_community.document_loaders import WebBaseLoader
desired_webpage_link = "https://lilianweng.github.io/posts/2023-06-23-agent/"
# the original tutorial only wanted us to keep html tags that
# contain any of the class names listed in the tuple assigned to the 
# class_ keyword, in the below variable. We discard all other html tags.
bs4_filtered = bs4.SoupStrainer()

webpage_loader = WebBaseLoader(
  web_paths=(desired_webpage_link,),
  bs_kwargs={"parse_only": bs4_filtered}
)

documents = webpage_loader.load()
#for key in documents[0]:
  #print(f"New key: {key}")
#print(f"how many characters are in our page: {len(documents[0].page_content)}")
#print(f"The first two thousand characters of the document: {documents[0].page_content[:2000]}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# create a text splitter instance
recursive_text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=200,
  add_start_index=True,
)

# obtain splitted text. our document is now a bunch of document shards.
splitted_text = recursive_text_splitter.split_documents(documents)
#print(f"The splits we got back:\n\n{splitted_text[4].page_content}")
#print(f"Number of splits:{len(splitted_text)}")

# We've loaded in our document. 
Split it up into text chunks, to allow our model to access bits of the data that fit into the LLM context window.

Now, it's time to put these chunks into a storage format the LLM can access.

In [9]:
# the vector store was previously initialized with our defined embeddings
# in one command sequence, we tell the vector store to embed each of our document chunks.
# we then store the embedded versions of these chunks into our vector store, for 
# later retrieval
vector_store__document_embedded_vectors = vector_store.add_documents(splitted_text)


In [10]:
print(f"vector store shape: {vector_store__document_embedded_vectors}")

vector store shape: ['27b0208f-fb09-41f6-946d-ce7c155e1a2c', '9dba033e-c50f-4604-a5ff-a0915a3f16a4', '74af6b07-6a62-4d5a-8a34-7799bbe0d8b0', 'd3636f85-94c7-4e72-93ee-980858edbc90', 'fda53b09-33c6-403c-8ba9-68e56f344a27', '4582af14-ab6d-42b0-a984-01f3a28739a1', '5ec743e3-f521-4118-9788-b8e338178237', '040ae68b-e89e-4c47-a1d4-f6dde175929a', '85c55627-1ad5-439c-8da4-cf7d9a9826b7', '6fad202c-ca86-499e-8c6d-9fa6221d3b53', '3f24a66b-e74b-4f86-8e84-caced4476401', 'dfe325e8-0ef1-482d-a693-dcd7cbad9fd9', '76962921-8f12-430b-9efd-2aec0980ac59', 'a174e7b2-5ffe-451a-bf18-9599f0012096', 'ebbd4003-7d36-4595-a2ec-643f9575a38f', '63089be8-ddcf-4907-a581-24c6f7d21a49', '3c560b62-350f-4c18-a28b-a41340defe53', '14952486-50f0-4823-b0a8-7f660e34e5fc', '36ed5da8-0c6a-43b3-b6c2-e39d0b1cf00a', 'd204562e-84b4-44b1-8927-bfc08dece243', 'cd2882a2-1847-4330-b643-95825c318bd2', '01011d21-8445-493b-97c3-ad656a5bc52c', '78785154-e9dc-4a80-bcb8-4685e017b06b', '1a707a5c-75d8-463d-8b41-5bc32170907d', 'd3a317ee-6803-4933

# We now should be able to answer User Queries
By accessing our vector store and returning the relevant chunks of text, from Lillian's blog post, that answers the user's questions.

## Onto Retrieval And Generation

In [11]:
# Get RAG prompt
from langchain import hub

# The actual RAG prompt
rag_prompt = hub.pull("rlm/rag-prompt")
#print(f"Actual template response: {rag_prompt.messages[0].prompt.template}.")

example_rag_message = rag_prompt.invoke({
  "context": "context here",
  "question": "question here"
}).to_messages() # Example of passing in context and question to a full RAG prompt.

print(f"Example rag message: {example_rag_message}")

Example rag message: [HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: question here \nContext: context here \nAnswer:", additional_kwargs={}, response_metadata={})]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
## Use LangGraph to generate for our RAG application.

# Begin the state definition phase, where we discuss what type of data we want our RAG model to process.

from langchain_core.documents import Document
from typing_extensions import List, TypedDict

# Create the actual state, to define what data our RAG model handles

class RAG_State(TypedDict):
  question: str
  context: List[Document] # remember, context should be several chunks of a document
  answer: str

def retrieve(rag_state: RAG_State):
  """
  Retrieves relevant data, based on our user question, from the vector store

  Args:
    rag_state: A dictionary which, for this problem, contains the user question, in the key 'question'. Has to be of type RAG_State
  
  Returns:
    All relevant context document chunks. Should be a dictionary of the following form: {str ; List[Document]}
  """
  similar_document_chunks = vector_store.similarity_search(rag_state["question"])
  return {"context" : similar_document_chunks}

def generate(rag_state: RAG_State):
  """
  Generate appropriate RAG response to the user query.

  Args:
    rag_state: A dictionary which, for this problem, contains the user question, in the key 'question', and the context of the question, in the key 'context'. Has to be of type RAG_State
  
  Returns:
    The model's response. Should be a dictionary of the following form: {str ; ?}
  """
  documents_contents = "\n\n".join(doc.page_content for doc in rag_state["context"])
  formalized_RAG_prompt = rag_prompt.invoke({"question": rag_state["question"], "context": rag_state["context"]})
  llm_RAG_response = llm.invoke(formalized_RAG_prompt)
  return {"answer" : llm_RAG_response.content}

In [13]:
# Create a graph object, to link together the retrieval and generate steps.
# Note that our retrieval and generate steps are simple as of now, but can be
# made more complex.

from langgraph.graph import START, StateGraph # START is a special Start Node

rag_agent_maker = StateGraph(RAG_State).add_sequence([retrieve, generate])
rag_agent_maker.add_edge(START, "retrieve") # links the special Start node to the retrieve node sequence we defined above.
rag_agent = rag_agent_maker.compile() # now we have made our RAG agent.

In [21]:
user_question = input("Please input your question to give to our RAG agent.")
rag_agent_response = rag_agent.invoke({"question": user_question})

print(f"Context: {rag_agent_response['context'][0].page_content}")
print(f"Answer: {rag_agent_response['answer']}")

Context: }
]
Then after these clarification, the agent moved into the code writing mode with a different system message.
System message:
Answer: I'm sorry, but the provided context does not contain information about choosing another item and repeating a process. Therefore, I cannot provide an answer to your question.
